In [19]:

# path

In [2]:
import fiona 
import pandas as pd 
import geopandas as gpd 
import branca
import random
import os
path = os.path.dirname(os.path.abspath(os.path.dirname(__name__)))
# import os

fiona.drvsupport.supported_drivers['libkml'] = 'rw' 
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

# Specify the path to our KMZ file
# path = os.path.abspath(__file__)
path = path + "/arquivos/"
kmz_file_path = path + "linhas_observatório.kmz"
kml_file_path = path + "linhas_tratadas.kml"

layers = gpd.list_layers(kml_file_path)

gdf_list = []
for layer in fiona.listlayers(kml_file_path):
    if layer == 'Linhas prontas':
        continue
    gdf = gpd.read_file(kml_file_path, driver='LIBKML', layer=layer)
    gdf_list.append(gdf)

gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

# linha_teste = gdf.iloc[1339]

# linha_teste

gdf_filtered = gdf.query("Name != ''")
gdf_filtered = gdf_filtered.query("Name != '1'")
gdf = gdf_filtered.query("Name != 'inativo'")
gdf

,Name,Description,geometry
1280,1501 - IDA,,"LINESTRING Z (-43.88157 -16.70074 0, -43.88143..."
1281,1501 - VOLTA,"13,4 km","LINESTRING Z (-43.82687 -16.73644 0, -43.82684..."
1282,1601 - IDA,"7, 66 km","LINESTRING Z (-43.8637 -16.69638 0, -43.86386 ..."
1283,1601 - VOLTA,"7,38","LINESTRING Z (-43.86828 -16.7425 0, -43.8687 -..."
1284,1701 - IDA,"11,5 km","LINESTRING Z (-43.8735 -16.67598 0, -43.87359 ..."
...,...,...,...
1337,7101 - VOLTA,"10,2","LINESTRING Z (-43.87352 -16.69642 0, -43.87342..."
1338,7103 - IDA,"16,7 km","LINESTRING Z (-43.88644 -16.79298 0, -43.8863 ..."
1339,7103 - VOLTA,"18,5","LINESTRING Z (-43.87403 -16.6892 0, -43.87448 ..."
1340,8401 - IDA,"13,8km","LINESTRING Z (-43.89212 -16.75349 0, -43.89248..."


In [27]:
import folium
import json
# from osgeo import gdal
# gdal.SetConfigOption('SHAPE_RESTORE_SHX', 'YES')

m = folium.Map([-16.737, -43.8647])

shapefile_moc = path + "bairros_sirgas.shp"

gdf_moc = gpd.read_file(shapefile_moc)

gdf_moc

DataSourceError: Unable to open /home/yagomaia/projetos/mestrado_amanda/arquivos/bairros_sirgas.shx or /home/yagomaia/projetos/mestrado_amanda/arquivos/bairros_sirgas.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.

In [3]:
print(path)
shapefile_ibge = path +"IBGE/" + "MG_Municipios_2022.shp"
gdf_ibge = gpd.read_file(shapefile_ibge)

gdf_ibge

/home/yagomaia/projetos/mestrado_amanda/arquivos/


,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,3100104,Abadia dos Dourados,MG,880.461,"POLYGON ((-47.45335 -18.46925, -47.45341 -18.4..."
1,3100203,Abaeté,MG,1817.067,"POLYGON ((-45.50026 -19.22585, -45.50024 -19.2..."
2,3100302,Abre Campo,MG,470.551,"POLYGON ((-42.42074 -20.39718, -42.42076 -20.3..."
3,3100401,Acaiaca,MG,101.886,"POLYGON ((-43.12231 -20.43351, -43.12239 -20.4..."
4,3100500,Açucena,MG,815.422,"POLYGON ((-42.43777 -19.17542, -42.43781 -19.1..."
...,...,...,...,...,...
848,3171808,Virginópolis,MG,439.878,"POLYGON ((-42.71402 -18.89438, -42.71448 -18.8..."
849,3171907,Virgolândia,MG,281.022,"POLYGON ((-42.30397 -18.53117, -42.30397 -18.5..."
850,3172004,Visconde do Rio Branco,MG,243.351,"POLYGON ((-42.85714 -21.08093, -42.85731 -21.0..."
851,3172103,Volta Grande,MG,205.552,"POLYGON ((-42.60347 -21.81934, -42.60342 -21.8..."


In [4]:
map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB Voyager')
folium.Rectangle(
    bounds=[[-90, -180], [90, 180]],
    color="#F9FFBC",
    fill=True,
    fill_color="#F9FFBC",
    fill_opacity=1
).add_to(m)
folium.TileLayer().add_to(m)

# Converta o GeoDataFrame para GeoJSON
# geojson_data = json.loads(gdf_moc.to_json())
# geojson_data
# # Adicione o GeoJSON ao mapa
folium.GeoJson(
    gdf_moc,
    style_function=lambda feature: {
        'fillColor': 'white',  # Cor de preenchimento do polígono
        'color': 'black',         # Cor das bordas
        'weight': 0.7,           # Espessura das bordas
        'fillOpacity': 0.5,      # Opacidade do preenchimento
    },
    name = 'Bairros'
).add_to(m)

m

C:\Users\Usuario\AppData\Local\Temp\ipykernel_129708\2158909185.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]


In [5]:
def randon_color():
    cor = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    # print(cor)
    return cor

In [6]:
def add_line_to_map(linha, geometry, group):
    cor = randon_color()
    folium.PolyLine(
        locations=[(lat, lon) for lon, lat in zip(geometry.xy[0], geometry.xy[1])],
        color= cor,
        weight=2.5,
        opacity=1,
        tooltip = linha
    ).add_to(group)

def add_line_to_map_sem_grupo(linha, geometry, cor):
    # cor = randon_color()
    folium.PolyLine(
        locations=[(lat, lon) for lon, lat in zip(geometry.xy[0], geometry.xy[1])],
        color= cor,
        weight=3,
        opacity=1,
        tooltip = linha
    ).add_to(m)

In [41]:
map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB Voyager')
# folium.Rectangle(
#     bounds=[[-90, -180], [90, 180]],
#     color="#F9FFBC",
#     fill=True,
#     fill_color="#F9FFBC",
#     fill_opacity=1
# ).add_to(m)
# folium.TileLayer().add_to(m)

# Converta o GeoDataFrame para GeoJSON
# geojson_data = json.loads(gdf_moc.to_json())
# geojson_data
# # Adicione o GeoJSON ao mapa
folium.GeoJson(
    gdf_moc,
    style_function=lambda feature: {
        'fillColor': 'white',  # Cor de preenchimento do polígono
        'color': 'black',         # Cor das bordas
        'weight': 0.7,           # Espessura das bordas
        'fillOpacity': 0.5,      # Opacidade do preenchimento
    },
    name = 'Bairros'
).add_to(m)

legenda_html = """
<div style="
     position: fixed; 
     top : 50px; right: 80px; width: 400px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <div>
        <h4 style="text-align: center; font-weight: bold;">LINHAS DE ÔNIBUS DE MONTES CLAROS - MG <br>(PADRÃO BOM)</h4>
     </div>
</div>
"""

legenda_html += """
     <div style="
     position: fixed; 
     top : 250px; right: 150px; width: 250px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Legenda</b><br>
     <div style="display: flex; justify-content: space-between;">
     <div style="flex: 1;">
"""

linhas_bom = ['1701', '1702', '2202', '2601', '2603', '4601', '4603', '5801', '5802', '7103']
cores = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'black', '#37FFEE', 'pink']
count = 0
for index, row in gdf.iterrows():
    separacao = row.Name.strip().split('-')
    if len(separacao) != 2:
        continue
    linha, sentido = separacao
    # print(sentido)
    if sentido.strip() == 'IDA' and linha.strip() in linhas_bom:
        cor = randon_color()
        # print(linha)
        add_line_to_map_sem_grupo(linha, row.geometry, cores[count])
        if count == len(linhas_bom)/2:
            legenda_html += '</div><div style="flex: 1;">'
        legenda_html += f"""
            <div style="display: flex; align-items: center; width: 120px; margin-bottom: 5px;">
                <div style="width: 40px; height: 10px; background-color: {cores[count]}; margin-right: 5px;"></div>
                {linha}
            </div>
        """
        count+=1
    # print(f"Frequência de Atendimento Linha {linha}: {calcular_indicadores.calcular_frequencia_atendimento(linha.strip())}")
    # if sentido.strip() == 'IDA':
    #     add_line_to_map('2601', row.geometry, ida_group)
    # elif sentido.strip() == 'VOLTA':
    #     add_line_to_map('2601', row.geometry, volta_group)
    # else:
    #     add_line_to_map_sem_grupo(linha, row.geometry)
legenda_html += "</div></div></div>"

C:\Users\Usuario\AppData\Local\Temp\ipykernel_129708\2698120230.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]


In [42]:

m.get_root().html.add_child(folium.Element(legenda_html))
m.save("mapa_com_legenda_linhas3.html")

import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('linhas_bom_2.png')

In [43]:
map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB Voyager')
# folium.Rectangle(
#     bounds=[[-90, -180], [90, 180]],
#     color="#F9FFBC",
#     fill=True,
#     fill_color="#F9FFBC",
#     fill_opacity=1
# ).add_to(m)
# folium.TileLayer().add_to(m)

# Converta o GeoDataFrame para GeoJSON
# geojson_data = json.loads(gdf_moc.to_json())
# geojson_data
# # Adicione o GeoJSON ao mapa
folium.GeoJson(
    gdf_moc,
    style_function=lambda feature: {
        'fillColor': 'white',  # Cor de preenchimento do polígono
        'color': 'black',         # Cor das bordas
        'weight': 0.7,           # Espessura das bordas
        'fillOpacity': 0.5,      # Opacidade do preenchimento
    },
    name = 'Bairros'
).add_to(m)

legenda_html = """
<div style="
     position: fixed; 
     top : 50px; right: 80px; width: 400px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <div>
        <h4 style="text-align: center; font-weight: bold;">LINHAS DE ÔNIBUS DE MONTES CLAROS - MG <br>(PADRÃO INSUFICIENTE)</h4>
     </div>
</div>
"""

legenda_html += """
     <div style="
     position: fixed; 
     top : 250px; right: 150px; width: 250px; height: auto; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     padding: 10px;
     ">
     <b>Legenda</b><br>
     <div style="display: flex; justify-content: space-between;">
     <div style="flex: 1;">
"""

linhas_insuficiente = ['1501', '1601', '2201', '2203', '2604', '3301', '4701', '5101', '5601', '5702', '5803', '5901', '5902', '6201R', '6202', '6404', '6604', '6901', '7101', '8201']
cores = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'black', '#37FFEE', 'pink', 'red', 'blue', 'green', 'purple', 'orange', 'brown', 'black', '#37FFEE', 'pink', 'red', 'blue', 'green', 'purple', 'orange', 'brown', 'black', '#37FFEE', 'pink']
count = 0
for index, row in gdf.iterrows():
    separacao = row.Name.strip().split('-')
    if len(separacao) != 2:
        continue
    linha, sentido = separacao
    # print(sentido)
    if sentido.strip() == 'IDA' and linha.strip() in linhas_insuficiente:
        cor = randon_color()
        print(linha)
        add_line_to_map_sem_grupo(linha, row.geometry, cores[count])
        if count == len(linhas_insuficiente)/2:
            legenda_html += '</div><div style="flex: 1;">'
        legenda_html += f"""
            <div style="display: flex; align-items: center; width: 120px; margin-bottom: 5px;">
                <div style="width: 40px; height: 10px; background-color: {cor}; margin-right: 5px;"></div>
                {linha}
            </div>
        """
        count+=1
    # print(f"Frequência de Atendimento Linha {linha}: {calcular_indicadores.calcular_frequencia_atendimento(linha.strip())}")
    # if sentido.strip() == 'IDA':
    #     add_line_to_map('2601', row.geometry, ida_group)
    # elif sentido.strip() == 'VOLTA':
    #     add_line_to_map('2601', row.geometry, volta_group)
    # else:
    #     add_line_to_map_sem_grupo(linha, row.geometry)
legenda_html += "</div></div></div>"

1501 
1601 
2201 
2203 
2604 
3301 
4701 
5101 
5601 
5702 
5803 
5901 
5902 
6201R 
6202 
6404 
6604 
6901 
7101 


C:\Users\Usuario\AppData\Local\Temp\ipykernel_129708\44800598.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]


In [44]:

m.get_root().html.add_child(folium.Element(legenda_html))
# m.save("mapa_com_legenda_linhas3.html")

import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('linhas_insuficiente_2.png')

m